# Rigid registration

In [1]:
import copy
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import logging

from opentps.core.processing.registration.registrationRigid import RegistrationRigid
from opentps.core.examples.syntheticData import *
from opentps.core.processing.imageProcessing.resampler3D import resampleImage3DOnImage3D
from opentps.core.processing.imageProcessing.imageTransform3D import rotateData, translateData


15/07/2024 03:33:09 PM - root - INFO - Loading logging configuration: C:\Users\cgaban\opentps\opentps_core\opentps\core\config\logger\logging_config.json
15/07/2024 03:33:09 PM - opentps.core._loggingConfig - INFO - Log level set: INFO
15/07/2024 03:33:11 PM - opentps.core.processing.imageProcessing.cupyImageProcessing - WARNING - Cannot import Cupy module
15/07/2024 03:33:13 PM - opentps.core.processing.registration.registrationMorphons - WARNING - cupy not found.
15/07/2024 03:33:13 PM - opentps.core.processing.registration.morphonsCupy - WARNING - cupy not found.
15/07/2024 03:33:13 PM - opentps.core.processing.C_libraries.libInterp3_wrapper - WARNING - cupy not found.


### Generate synthetic images

In [3]:
fixed = createSynthetic3DCT()
moving = copy.copy(fixed)

translation = np.array([15, 0, 10])
rotation = np.array([0, 5, 0])

translateData(moving, translation, outputBox='same')
rotateData(moving, rotation, outputBox='same')

AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

###  Perform the Rigid registration

In [4]:
start_time = time.time()
reg = RegistrationRigid(fixed, moving)
transform = reg.compute()

processing_time = time.time() - start_time
print('Registration processing time was', processing_time, '\n')
print('Translation', transform.getTranslation())
print('Rotation in deg', transform.getRotationAngles(inDegrees=True), '\n')

## Two ways of getting the deformed moving image
#deformedImage = reg.deformed
deformedImage = transform.deformImage(moving)

## Resample it to the same grid as the fixed image
resampledOnFixedGrid = resampleImage3DOnImage3D(deformedImage, fixedImage=fixed, fillValue=-1000)

Final metric value: 0.0
Optimizer's stopping condition, RegularStepGradientDescentOptimizerv4: Gradient magnitude tolerance met after 0 iterations. Gradient magnitude (0) is less than gradient magnitude tolerance (0.0001).
itk::simple::CompositeTransform
 CompositeTransform (000001FFA31879C0)
   RTTI typeinfo:   class itk::CompositeTransform<double,3>
   Reference Count: 1
   Modified Time: 332808
   Debug: Off
   Object Name: 
   Observers: 
     none
   Transforms in queue, from begin to end:
   >>>>>>>>>
   Euler3DTransform (000001FFB4321780)
     RTTI typeinfo:   class itk::Euler3DTransform<double>
     Reference Count: 1
     Modified Time: 332799
     Debug: Off
     Object Name: 
     Observers: 
       none
     Matrix: 
       1 0 0 
       0 1 0 
       0 0 1 
     Offset: [0, 0, 0]
     Center: [84.5, 84.5, 99]
     Translation: [0, 0, 0]
     Inverse: 
       1 0 0 
       0 1 0 
       0 0 1 
     Singular: 0
     Euler's angles: AngleX=0 AngleY=0 AngleZ=0
     m_ComputeZY

### Compute the difference between the 2 images and check results of the registration

In [5]:
#IMAGE DIFFERENCE
diff_before = fixed.copy()
diff_before._imageArray = fixed.imageArray - moving.imageArray
diff_after = fixed.copy()
diff_after._imageArray = fixed.imageArray - resampledOnFixedGrid.imageArray

# CHECK RESULTS
diff_before_sum = abs(diff_before.imageArray).sum()
diff_after_sum = abs(diff_after.imageArray).sum()
assert diff_before_sum - diff_after_sum > 0, f"Image difference is larger after registration"

AssertionError: Image difference is larger after registration

### Plot results

In [6]:
#Output path
output_path = 'Output'
if not os.path.exists(output_path):
    os.makedirs(output_path)

y_slice = 95
fig, ax = plt.subplots(2, 3,figsize=(8,10))
ax[0, 0].imshow(fixed.imageArray[:, y_slice, :],cmap='gray')
ax[0, 0].set_title('Fixed')
ax[0, 0].set_xlabel('Origin: '+f'{fixed.origin[0]}'+','+f'{fixed.origin[1]}'+','+f'{fixed.origin[2]}')
ax[0, 1].imshow(moving.imageArray[:, y_slice, :],cmap='gray')
ax[0, 1].set_title('Moving')
ax[0, 1].set_xlabel('Origin: ' + f'{moving.origin[0]}' + ',' + f'{moving.origin[1]}' + ',' + f'{moving.origin[2]}')
diffBef = ax[0, 2].imshow(abs(diff_before.imageArray[:, y_slice, :]), vmin=0, vmax=1500)
ax[0, 2].set_title('Diff before')
fig.colorbar(diffBef, ax=ax[0, 2])
ax[1, 0].imshow(deformedImage.imageArray[:, y_slice, :],cmap='gray')
ax[1, 0].set_title('DeformedMoving')
ax[1, 0].set_xlabel('Origin: ' + f'{deformedImage.origin[0]:.1f}' + ',' + f'{deformedImage.origin[1]:.1f}' + ',' + f'{deformedImage.origin[2]:.1f}')
ax[1, 1].imshow(resampledOnFixedGrid.imageArray[:, y_slice, :],cmap='gray')
ax[1, 1].set_title('resampledOnFixedGrid')
ax[1, 1].set_xlabel('Origin: ' + f'{resampledOnFixedGrid.origin[0]:.1f}' + ',' + f'{resampledOnFixedGrid.origin[1]:.1f}' + ',' + f'{resampledOnFixedGrid.origin[2]:.1f}')
diffAft = ax[1, 2].imshow(abs(diff_after.imageArray[:, y_slice, :]), vmin=0, vmax=1500)
ax[1, 2].set_title('Diff after')
plt.show()
plt.savefig(os.path.join(output_path,"Rigid1.png"),format = 'png')
plt.close()

![png](\assets\img_notebooks\Rigid1.png)